In [1]:
import torch
import torchvision
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

from ultralytics import YOLO

In [4]:
image_folder = 'C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1'  # Replace with your images folder path
label_folder = 'C:/Users/Admin/Desktop/Model folder/Pod Recognition/labels/train'  # Replace with your labels folder path

# List all image files (assuming image extensions are .jpg, .png, etc.)
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# List all label files (assuming labels are in .txt format)
label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

# Remove file extensions from both lists to compare them
image_names = [os.path.splitext(f)[0] for f in image_files]
label_names = [os.path.splitext(f)[0] for f in label_files]

# Check for images without labels
images_without_labels = [img for img in image_names if img not in label_names]

# Check for labels without corresponding images
labels_without_images = [lbl for lbl in label_names if lbl not in image_names]

# Output the results
if len(images_without_labels) == 0 and len(labels_without_images) == 0:
    print("All images have corresponding labels, and vice versa.")
else:
    if len(images_without_labels) > 0:
        print(f"Images without labels: {images_without_labels}")
    if len(labels_without_images) > 0:
        print(f"Labels without corresponding images: {labels_without_images}")

All images have corresponding labels, and vice versa.


In [3]:

image_folder = 'C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1'  # Replace with your images folder path
label_folder = 'C:/Users/Admin/Desktop/Model folder/Pod Recognition/labels/train'
# List all image files (assuming image extensions are .jpg, .png, etc.)
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# List all label files (assuming labels are in .txt format)
label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

# Remove file extensions from both lists to compare them
image_names = [os.path.splitext(f)[0] for f in image_files]
label_names = [os.path.splitext(f)[0] for f in label_files]

# Find images without corresponding labels
images_without_labels = [img for img in image_names if img not in label_names]

# Delete images without labels
for img in images_without_labels:
    img_file = img + '.jpg'  # Adjust extension if needed
    img_path = os.path.join(image_folder, img_file)
    if os.path.exists(img_path):
        os.remove(img_path)
        print(f"Deleted: {img_path}")

print(f"Deleted {len(images_without_labels)} images without labels.")

Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_120853.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_120956.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_121012.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_121302.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_121341.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_121530.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_121548.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_121736.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_121749.jpg
Deleted: C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1\IMG_20240804_121910.jpg


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths to your dataset
image_folder = 'C:/Users/Admin/Desktop/Model folder/Pod Recognition/dataset comp 1'  # Replace with your images folder path
label_folder = 'C:/Users/Admin/Desktop/Model folder/Pod Recognition/labels/train'  # Path to images

# Create the structure for YOLOv8
base_dir = 'C:/Users/Admin/Desktop/Model folder/yolo_pod'  # Change this path if necessary
os.makedirs(f'{base_dir}/images/train', exist_ok=True)
os.makedirs(f'{base_dir}/images/val', exist_ok=True)
os.makedirs(f'{base_dir}/images/test', exist_ok=True)
os.makedirs(f'{base_dir}/labels/train', exist_ok=True)
os.makedirs(f'{base_dir}/labels/val', exist_ok=True)
os.makedirs(f'{base_dir}/labels/test', exist_ok=True)

# Get list of image files and corresponding label files
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
label_files = [f.replace('.jpg', '.txt').replace('.png', '.txt') for f in image_files]

# Split dataset into train (80%) and remaining (20%)
train_images, remaining_images = train_test_split(image_files, test_size=0.2, random_state=42)

# Split the remaining dataset into validation (50% of the remaining 20%) and test (50% of the remaining 20%)
val_images, test_images = train_test_split(remaining_images, test_size=0.5, random_state=42)

# Move training images and labels
for img_file in train_images:
    label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
    # Move image
    shutil.copy(os.path.join(image_folder, img_file), os.path.join(base_dir, 'images/train', img_file))
    # Move label
    if os.path.exists(os.path.join(label_folder, label_file)):
        shutil.copy(os.path.join(label_folder, label_file), os.path.join(base_dir, 'labels/train', label_file))

# Move validation images and labels
for img_file in val_images:
    label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
    # Move image
    shutil.copy(os.path.join(image_folder, img_file), os.path.join(base_dir, 'images/val', img_file))
    # Move label
    if os.path.exists(os.path.join(label_folder, label_file)):
        shutil.copy(os.path.join(label_folder, label_file), os.path.join(base_dir, 'labels/val', label_file))

# Move test images and labels
for img_file in test_images:
    label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
    # Move image
    shutil.copy(os.path.join(image_folder, img_file), os.path.join(base_dir, 'images/test', img_file))
    # Move label
    if os.path.exists(os.path.join(label_folder, label_file)):
        shutil.copy(os.path.join(label_folder, label_file), os.path.join(base_dir, 'labels/test', label_file))

print('Dataset successfully structured for YOLOv8 with 80% training, 10% validation, and 10% test splits!')

Dataset successfully structured for YOLOv8 with 80% training, 10% validation, and 10% test splits!


In [6]:
model_save_path = "C:/Users/Admin/Desktop/Model folder/yolo_models"

In [7]:
# Load a pre-trained model and update it with your dataset
model = YOLO('yolov8n.pt')  # Load YOLOv8n pre-trained model (adjust if needed)

# Train the model
model.train(data='C:/Users/Admin/Desktop/Model folder/yolo_pod/data.yaml', epochs=10, batch=32, project=model_save_path, name='yolov8_pod_model')

Ultralytics YOLOv8.2.98  Python-3.8.5 torch-2.4.1+cpu CPU (Intel Core(TM) i7-8650U 1.90GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/Admin/Desktop/Model folder/yolo_pod/data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=C:/Users/Admin/Desktop/Model folder/yolo_models, name=yolov8_pod_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sa

train: Scanning C:\Users\Admin\Desktop\Model folder\yolo_pod\labels\train...:   0%|          | 0/500 [00:00<?, ?it/s]C:\Users\Admin\anaconda3\lib\site-packages\PIL\Image.py:2834: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
train: Scanning C:\Users\Admin\Desktop\Model folder\yolo_pod\labels\train... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1393.80it/s]


train: New cache created: C:\Users\Admin\Desktop\Model folder\yolo_pod\labels\train.cache


val: Scanning C:\Users\Admin\Desktop\Model folder\yolo_pod\labels\val... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<00:00, 1426.85it/s]

val: New cache created: C:\Users\Admin\Desktop\Model folder\yolo_pod\labels\val.cache


...

                   all         62        175      0.441      0.686      0.487       0.19



10 epochs completed in 2.200 hours.
Optimizer stripped from C:\Users\Admin\Desktop\Model folder\yolo_models\yolov8_pod_model\weights\last.pt, 6.2MB
Optimizer stripped from C:\Users\Admin\Desktop\Model folder\yolo_models\yolov8_pod_model\weights\best.pt, 6.2MB

Validating C:\Users\Admin\Desktop\Model folder\yolo_models\yolov8_pod_model\weights\best.pt...
Ultralytics YOLOv8.2.98  Python-3.8.5 torch-2.4.1+cpu CPU (Intel Core(TM) i7-8650U 1.90GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:55<00:00, 55.19s/it]


                   all         62        175      0.441      0.697      0.488       0.19
Speed: 3.3ms preprocess, 136.3ms inference, 0.0ms loss, 8.6ms postprocess per image
Results saved to C:\Users\Admin\Desktop\Model folder\yolo_models\yolov8_pod_model


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002910D9C68E0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [9]:

model = YOLO("C:/Users/Admin/Desktop/Model folder/yolo_models/yolov8_pod_model/weights/best.pt")

# Perform inference on an image
results = model.predict(source = "C:/Users/Admin/Desktop/Model folder/yolo_pod/images/test", save=True)



image 1/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\test\IMG_20240804_120926.jpg: 640x480 4 pods, 242.2ms
image 2/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\test\IMG_20240804_123927.jpg: 640x640 2 pods, 359.9ms
image 3/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\test\IMG_20240804_124959.jpg: 640x480 4 pods, 187.6ms
image 4/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\test\IMG_20240804_125333.jpg: 640x480 3 pods, 142.4ms
image 5/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\test\IMG_20240804_130054.jpg: 480x640 1 pod, 199.8ms
image 6/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\test\IMG_20240804_130146.jpg: 640x480 2 pods, 143.9ms
image 7/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\test\IMG_20240804_170125.jpg: 640x480 4 pods, 169.6ms
image 8/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\test\IMG_20240804_170137.jpg: 640x480 2 pods, 204.0ms
image 9/63 C:\Users\Admin\Desktop\Model folder\yolo_pod\images\t